In [35]:
import glob
import re
import pandas as pd
from IPython.core.display import display
from pathlib import Path

data_path = '../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/cctv'

In [99]:
coref_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Sentence_lengths', 'Coref_Clusters'])
print(coref_df)

Empty DataFrame
Columns: [Doc_ID, Text_ID, Text, Sentence_lengths, Coref_Clusters]
Index: []


In [97]:
def get_coref_clusters(path, document_id):
    temp_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Sentence_lengths', 'Coref_Clusters'])

    with open(path, 'r') as file:
        lines = file.readlines()

        sentence = []
        text = []
        sentence_lengths = []
        coref_clusters = {}
        temp_clusters = {}
        columns = []

        for line in lines:
            # if we enter a new document
            if line.startswith('#'):
                if text:
                    document_id = columns[0]
                    text_id = columns[1]
                    temp_df = temp_df.append({'Doc_ID': document_id,
                                              'Text_ID': document_id + '-' + str(text_id),
                                              'Text': text,
                                              'Sentence_lengths': sentence_lengths,
                                              'Coref_Clusters': list(coref_clusters.values())},
                                             ignore_index=True)
                    text = []
                    sentence_lengths = []
                    coref_clusters = {}
                    temp_clusters = {}
            elif line == '\n':
                text.append(sentence)
                sentence_lengths.append(int(columns[2]) + 1) #zero indexed in dataset
                sentence = []
            else:
                columns = line.split()
                index = int(columns[2]) + sum(sentence_lengths)
                sentence.append(columns[3])
                coref_column = columns[-1]
                if not coref_column == '-':
                    corefs = coref_column.split('|')
                    for coref in corefs:
                        key = coref.strip('()')
                        if re.match(r'\([0-9]+\)', coref):
                            coref_clusters.setdefault(key, []).append([index, index])
                        if re.match(r'^\([0-9]+$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                        if re.match(r'^[0-9]+\)$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                            cluster_span = temp_clusters.pop(key)
                            coref_clusters.setdefault(key, []).append(cluster_span)
    display(temp_df)
    return temp_df

In [100]:

for path in Path(data_path).rglob('*.gold_conll'):
    print(path)
    filename = str(path).split('/')[-1]
    #document_id = filename.split('.')[0]
    df = get_coref_clusters(path, filename)
    coref_df = coref_df.append(df)

../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/cctv/00/cctv_0005.gold_conll
[14, 22, 29, 11, 5, 6, 12, 5]
[5, 5, 22, 37, 11, 13, 18, 15, 35, 19, 26, 2, 13, 2, 13, 26, 23, 2, 2, 18, 24, 10, 8, 6, 7, 8, 2, 29, 20, 6, 2, 20, 39, 2, 15, 36, 4, 4]
[12, 11, 70, 15, 18, 38, 22, 22, 2, 17, 2, 11, 4, 6, 30, 2, 17, 2, 29, 35, 2, 29, 17]
[11, 21, 2, 26, 39, 6, 2, 32, 36, 2, 14, 27, 2, 2, 11, 44, 2, 13, 9, 29, 5, 19, 22, 41, 13, 9, 2, 2, 8]
[8, 11, 4, 8, 13, 8, 4, 6, 4, 4, 17, 3, 6, 8, 12, 14, 8, 11, 13, 8, 21, 44, 13, 42, 34, 21, 13, 2, 18, 15, 20]
[32, 4, 9, 17, 27, 19, 14, 52, 2, 15, 37, 41, 21, 34, 64, 2, 12, 12, 2, 7, 6, 10, 9, 29, 7, 2, 15, 12, 23, 19, 39, 2, 11, 16]
[32, 12, 22, 8, 2, 66, 16, 2, 22, 42, 17, 2, 41, 2, 24, 21, 13, 2, 7, 2, 9, 2]
[2, 15, 23, 8, 14, 27, 28, 21, 21, 49, 11, 13, 10, 15, 14, 57, 35, 35]
[50, 2, 4, 16, 22, 47, 32, 14, 9, 30, 43, 33, 13, 14, 2, 17]
[2, 21, 2, 7, 13, 50, 36, 4, 24, 24, 16, 43, 23, 16]
[

,Doc_ID,Text_ID,Text,Sentence_lengths,Coref_Clusters
0,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-0,"[[--, basically, ,, it, was, unanimously, agre...","[14, 22, 29, 11, 5, 6, 12, 5]","[[[16, 16], [19, 23]], [[25, 27], [42, 44], [5..."
1,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-1,"[[Hello, ,, dear, viewers, .], [Welcome, to, F...","[5, 5, 22, 37, 11, 13, 18, 15, 35, 19, 26, 2, ...","[[[7, 8], [233, 234]], [[10, 10], [172, 172]],..."
2,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-2,"[[Well, ,, what, in, fact, was, the, content, ...","[12, 11, 70, 15, 18, 38, 22, 22, 2, 17, 2, 11,...","[[[9, 10], [25, 26], [103, 104], [116, 117], [..."
3,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-3,"[[We, usually, make, it, sound, like, e-govern...","[11, 21, 2, 26, 39, 6, 2, 32, 36, 2, 14, 27, 2...","[[[17, 17], [22, 22]], [[29, 30], [42, 43]], [..."
4,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-4,"[[So, ,, how, exactly, did, citizens, react, ?...","[8, 11, 4, 8, 13, 8, 4, 6, 4, 4, 17, 3, 6, 8, ...","[[[23, 23], [33, 33], [45, 45], [74, 74]], [[2..."
5,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-5,"[[Well, ,, ah, ,, Yang, Yang, ,, you, work, at...","[32, 4, 9, 17, 27, 19, 14, 52, 2, 15, 37, 41, ...","[[[4, 5], [7, 7], [83, 83], [94, 94], [333, 33..."
6,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-6,"[[Yes, ,, many, ,, viewers, not, in, Beijing, ...","[32, 12, 22, 8, 2, 66, 16, 2, 22, 42, 17, 2, 4...","[[[7, 7], [171, 172]], [[28, 29], [258, 261], ..."
7,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-7,"[[Uh-huh, .], [Well, ,, what, measures, did, t...","[2, 15, 23, 8, 14, 27, 28, 21, 21, 49, 11, 13,...","[[[7, 10], [30, 38], [50, 53], [72, 72], [149,..."
8,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-8,"[[Ah, ,, it, can, be, said, ,, ah, ,, Yang, Ya...","[50, 2, 4, 16, 22, 47, 32, 14, 9, 30, 43, 33, ...","[[[9, 10], [58, 58], [66, 66], [226, 226], [27..."
9,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-9,"[[Uh-huh, .], [Well, ,, Professor, Zhou, ,, fr...","[2, 21, 2, 7, 13, 50, 36, 4, 24, 24, 16, 43, 2...","[[[4, 5], [242, 242]], [[41, 43], [209, 209], ..."


In [101]:
display(coref_df)
coref_df.to_pickle('data/coref_ontonotes.pkl')

,Doc_ID,Text_ID,Text,Sentence_lengths,Coref_Clusters
0,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-0,"[[--, basically, ,, it, was, unanimously, agre...","[14, 22, 29, 11, 5, 6, 12, 5]","[[[16, 16], [19, 23]], [[25, 27], [42, 44], [5..."
1,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-1,"[[Hello, ,, dear, viewers, .], [Welcome, to, F...","[5, 5, 22, 37, 11, 13, 18, 15, 35, 19, 26, 2, ...","[[[7, 8], [233, 234]], [[10, 10], [172, 172]],..."
2,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-2,"[[Well, ,, what, in, fact, was, the, content, ...","[12, 11, 70, 15, 18, 38, 22, 22, 2, 17, 2, 11,...","[[[9, 10], [25, 26], [103, 104], [116, 117], [..."
3,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-3,"[[We, usually, make, it, sound, like, e-govern...","[11, 21, 2, 26, 39, 6, 2, 32, 36, 2, 14, 27, 2...","[[[17, 17], [22, 22]], [[29, 30], [42, 43]], [..."
4,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-4,"[[So, ,, how, exactly, did, citizens, react, ?...","[8, 11, 4, 8, 13, 8, 4, 6, 4, 4, 17, 3, 6, 8, ...","[[[23, 23], [33, 33], [45, 45], [74, 74]], [[2..."
5,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-5,"[[Well, ,, ah, ,, Yang, Yang, ,, you, work, at...","[32, 4, 9, 17, 27, 19, 14, 52, 2, 15, 37, 41, ...","[[[4, 5], [7, 7], [83, 83], [94, 94], [333, 33..."
6,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-6,"[[Yes, ,, many, ,, viewers, not, in, Beijing, ...","[32, 12, 22, 8, 2, 66, 16, 2, 22, 42, 17, 2, 4...","[[[7, 7], [171, 172]], [[28, 29], [258, 261], ..."
7,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-7,"[[Uh-huh, .], [Well, ,, what, measures, did, t...","[2, 15, 23, 8, 14, 27, 28, 21, 21, 49, 11, 13,...","[[[7, 10], [30, 38], [50, 53], [72, 72], [149,..."
8,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-8,"[[Ah, ,, it, can, be, said, ,, ah, ,, Yang, Ya...","[50, 2, 4, 16, 22, 47, 32, 14, 9, 30, 43, 33, ...","[[[9, 10], [58, 58], [66, 66], [226, 226], [27..."
9,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-9,"[[Uh-huh, .], [Well, ,, Professor, Zhou, ,, fr...","[2, 21, 2, 7, 13, 50, 36, 4, 24, 24, 16, 43, 2...","[[[4, 5], [242, 242]], [[41, 43], [209, 209], ..."


../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/cctv/00/cctv_0005.gold_conll
../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/phoenix/00/phoenix_0003.gold_conll
../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/cnn/00/cnn_0008.gold_conll
../conll-formatted-ontonotes-5.0/conll-formatted-ontonotes-5.0/data/conll-2012-test/data/english/annotations/bc/msnbc/00/msnbc_0007.gold_conll
